# DICE - Notebook 3.2 - Model Training and Transfer Learning - Augmented Data

<br/>

```
*************************************************************************
**
** 2017 Mai 23
**
** In place of a legal notice, here is a blessing:
**
**    May you do good and not evil.
**    May you find forgiveness for yourself and forgive others.
**    May you share freely, never taking more than you give.
**
*************************************************************************
```

<table style="width:100%; font-size:14px; margin: 20px 0;">
    <tr>
        <td style="text-align:center">
            <b>Contact: </b><a href="mailto:contact@jonathandekhtiar.eu" target="_blank">contact@jonathandekhtiar.eu</a>
        </td>
        <td style="text-align:center">
            <b>Twitter: </b><a href="https://twitter.com/born2data" target="_blank">@born2data</a>
        </td>
        <td style="text-align:center">
            <b>Tech. Blog: </b><a href="http://www.born2data.com/" target="_blank">born2data.com</a>
        </td>
    </tr>
    <tr>
        <td style="text-align:center">
            <b>Personal Website: </b><a href="http://www.jonathandekhtiar.eu" target="_blank">jonathandekhtiar.eu</a>
        </td>
        <td style="text-align:center">
            <b>RSS Feed: </b><a href="https://www.feedcrunch.io/@dataradar/" target="_blank">FeedCrunch.io</a>
        </td>
        <td style="text-align:center">
            <b>LinkedIn: </b><a href="https://fr.linkedin.com/in/jonathandekhtiar" target="_blank">JonathanDEKHTIAR</a>
        </td>
    </tr>
</table>

## Objectives

This notebook aims to preprocess and prepare the dataset for later used during the training phase. 

There exists many methods to feed data into a Deep Learning with [Tensorflow](https://www.tensorflow.org/), the Python Library we have chosen to use for this study:

1. **From Disk**: Data can be inputed into a model with the **feed_dict** argument when running a *training operation*. It would  definitely be possible, however this process can be slow if there are a lot of data to read simultaneously and could be too large to be held in the GPU Memory.
<br><br>
2. **From a CSV File**: This [type of file](https://en.wikipedia.org/wiki/Comma-separated_values) is not revelant when dealing with images.
<br><br>
3. **From a preprocessed binary file**: Tensorflow is able to save and recover data in a binary format called [TFRecords](https://www.tensorflow.org/api_guides/python/python_io#TFRecords_Format_Details). The data can be preprocessed beforehand and only the necessary data can be saved and read in real time during the training. This approach is the fatest and most memory-efficient when dealing with images.

This notebook will focus on generating the necessary **TFRecord** files. Generating **TFRecords** is less intuitive than 
[HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format), used in other Deep Learning libraries such as [Keras](https://keras.io/). Using **TFRecords** will give you access to natively available tools, such as *Queue Runners*, *Coordinators*, *Supervisors*, *etc.*, to design [data pipelines](https://www.tensorflow.org/programmers_guide/reading_data) and process the images in a batch fashion.

This notebook will use [Tensorflow-Slim](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/slim) to ease the understanding and reduce the code complexity.

As we aim to to later re-train a CNN Model

This will be used later to retrain an CNN model: [Inception-V4](https://arxiv.org/abs/1602.07261) model developed by Szegedy et al. The model has been Pre-Trained with the [ImageNet](http://www.image-net.org/) dataset allowing a much more accurate result due to the large number of data avaiable in this dataset. We call this kind of process: "*Transfer Learning*".


This notebook will also randomly split the available data into two sets of data: [Training and Validation sets](https://stats.stackexchange.com/questions/19048/what-is-the-difference-between-test-set-and-validation-set). This process aims to reduce the [overfit](http://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html) of the model and thus improving its accuracy on previously unseen data. 

In this study the selection ratio has been chosen as followed:
- *training set:* 60%
- *validation set:* 40%.

---

As reminder before starting, the data have already been preprocessed (resized, augmented, etc.) in the first Notebook: **[DICE - Notebook 1 - Dataset Augmentation](https://github.com/DEKHTIARJonathan/DICE-DMU_Imagery_Classification_Engine/blob/master/DICE%20-%20Notebook%201%20-%20Dataset%20Augmentation.ipynb)**

The preprocessed data all have been saved as **JPEG images** and thus we will only focus on these data.

## 1. Notebook Initialisation

### 1.1. Load the necessary libraries

In [1]:
import os, time, math

import numpy as np

import tensorflow as tf
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
from tensorflow.python.platform import tf_logging as logging

from inception_files.preprocessing import inception_preprocessing
from inception_files.nets.inception_v1 import inception_v1, inception_v1_arg_scope

slim = tf.contrib.slim

In [2]:
#================ DATASET INFORMATION ======================
#State dataset directory where the tfrecord files are located
dataset_dir = 'data_prepared/'

#State where your log file is at. If it doesn't exist, create it.
log_dir = 'output/cleaned/'

#Create the log directory here. Must be done here otherwise import will activate this unneededly.
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

#State where your checkpoint file is
checkpoint_file = 'inception_files/models/inception_v1.ckpt'

if not os.path.isfile(checkpoint_file):
    raise Exception("The Inception Model does not exists")

#State the image size you're resizing your images to. We will use the default inception size of 299.
image_size = inception_v1.default_image_size

#State the number of classes to predict:
num_classes = 5

#State the labels file and read it
labels_file = 'data_prepared/labels.txt'

if not os.path.isfile(labels_file):
    raise Exception("The Label File does not exists")
else:
    labels = open(labels_file, 'r')

#Create a dictionary to refer each label to their string name
labels_to_name = {}
for line in labels:
    label, string_name = line.split(':')
    string_name = string_name[:-1] #Remove newline
    labels_to_name[int(label)] = string_name

#Create the file pattern of your TFRecord files so that it could be recognized later on
tf_record_start_name = 'dmunet_cleaned_dataset_'
file_pattern = tf_record_start_name + '%s_*.tfrecord'

#Create a dictionary that will help people understand your dataset better. This is required by the Dataset class later.
items_to_descriptions = {
    'image': 'A 3-channel RGB coloured flower image that is either tulips, sunflowers, roses, dandelion, or daisy.',
    'label': 'A label that is as such -- 0:daisy, 1:dandelion, 2:roses, 3:sunflowers, 4:tulips'
}

tf.logging.set_verbosity(tf.logging.INFO) #Set the verbosity to INFO level

#================= TRAINING INFORMATION ==================
#State the number of epochs to train
num_epochs = 30

#State your batch size => Choose the highest value which doesn't give you a memory error.
batch_size = 70

#Learning rate information and configuration (Up to you to experiment)
initial_learning_rate = 1e-4
learning_rate_decay_factor = 0.7
num_epochs_before_decay = 2

In [3]:
#============== DATASET LOADING ======================
#We now create a function that creates a Dataset class which will give us many TFRecord files to feed in the examples into a queue in parallel.
def get_split(split_name, dataset_dir, file_pattern=file_pattern):
    '''
    Obtains the split - training or validation - to create a Dataset class for feeding the examples into a queue later on. This function will
    set up the decoder and dataset information all into one Dataset class so that you can avoid the brute work later on.
    Your file_pattern is very important in locating the files later. 

    INPUTS:
    - split_name(str): 'train' or 'validation'. Used to get the correct data split of tfrecord files
    - dataset_dir(str): the dataset directory where the tfrecord files are located
    - file_pattern(str): the file name structure of the tfrecord files in order to get the correct data

    OUTPUTS:
    - dataset (Dataset): A Dataset class object where we can read its various components for easier batch creation later.
    '''

    #First check whether the split_name is train or validation
    if split_name not in ['train', 'validation']:
        raise ValueError('The split_name %s is not recognized. Please input either train or validation as the split_name' % (split_name))

    #Create the full path for a general file_pattern to locate the tfrecord_files
    file_pattern_path = os.path.join(dataset_dir, file_pattern % (split_name))

    #Count the total number of examples in all of these shard
    num_samples = 0
    file_pattern_for_counting = tf_record_start_name + split_name
    tfrecords_to_count = [os.path.join(dataset_dir, file) for file in os.listdir(dataset_dir) if file.startswith(file_pattern_for_counting)]
    for tfrecord_file in tfrecords_to_count:
        for record in tf.python_io.tf_record_iterator(tfrecord_file):
            num_samples += 1

    #Create a reader, which must be a TFRecord reader in this case
    reader = tf.TFRecordReader

    #Create the keys_to_features dictionary for the decoder
    keys_to_features = {
      'image/encoded': tf.FixedLenFeature((), tf.string, default_value=''),
      'image/format': tf.FixedLenFeature((), tf.string, default_value='jpg'),
      'image/class/label': tf.FixedLenFeature(
          [], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
    }

    #Create the items_to_handlers dictionary for the decoder.
    items_to_handlers = {
    'image': slim.tfexample_decoder.Image(),
    'label': slim.tfexample_decoder.Tensor('image/class/label'),
    }

    #Start to create the decoder
    decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)

    #Create the labels_to_name file
    labels_to_name_dict = labels_to_name

    #Actually create the dataset
    dataset = slim.dataset.Dataset(
        data_sources = file_pattern_path,
        decoder = decoder,
        reader = reader,
        num_readers = 4,
        num_samples = num_samples,
        num_classes = num_classes,
        labels_to_name = labels_to_name_dict,
        items_to_descriptions = items_to_descriptions)

    return dataset

In [4]:
def load_batch(dataset, batch_size, height=image_size, width=image_size, is_training=True):
    '''
    Loads a batch for training.

    INPUTS:
    - dataset(Dataset): a Dataset class object that is created from the get_split function
    - batch_size(int): determines how big of a batch to train
    - height(int): the height of the image to resize to during preprocessing
    - width(int): the width of the image to resize to during preprocessing
    - is_training(bool): to determine whether to perform a training or evaluation preprocessing

    OUTPUTS:
    - images(Tensor): a Tensor of the shape (batch_size, height, width, channels) that contain one batch of images
    - labels(Tensor): the batch's labels with the shape (batch_size,) (requires one_hot_encoding).

    '''
    #First create the data_provider object
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset,
        common_queue_capacity = 24 + 3 * batch_size,
        common_queue_min = 24)

    #Obtain the raw image using the get method
    raw_image, label = data_provider.get(['image', 'label'])

    #Perform the correct preprocessing for this image depending if it is training or evaluating
    image = inception_preprocessing.preprocess_image(raw_image, height, width, is_training)

    #As for the raw images, we just do a simple reshape to batch it up
    raw_image = tf.expand_dims(raw_image, 0)
    raw_image = tf.image.resize_nearest_neighbor(raw_image, [height, width])
    raw_image = tf.squeeze(raw_image)

    #Batch up the image by enqueing the tensors internally in a FIFO queue and dequeueing many elements with tf.train.batch.
    images, raw_images, labels = tf.train.batch(
        [image, raw_image, label],
        batch_size = batch_size,
        num_threads = 4,
        capacity = 4 * batch_size,
        allow_smaller_final_batch = True)

    return images, raw_images, labels

### Loading dataset and data batches

In [5]:
dataset = get_split('train', dataset_dir, file_pattern=file_pattern)
images, _, labels = load_batch(dataset, batch_size=batch_size)

num_batches_per_epoch = tf.cast(tf.ceil(dataset.num_samples / batch_size), tf.int32)

#Know the number steps to take before decaying the learning rate and batches per epoch
num_steps_per_epoch = num_batches_per_epoch #Because one step is one batch processed
decay_steps = tf.cast(num_epochs_before_decay * num_steps_per_epoch, tf.int32)

In [6]:
#Create the model inference
with slim.arg_scope(inception_v1_arg_scope()):
    logits, end_points = inception_v1(images, num_classes = dataset.num_classes, is_training = True)

In [7]:
#Define the scopes that you want to exclude for restoration
exclude = ["InceptionV1/Logits", "InceptionV1/AuxLogits"]
variables_to_restore = slim.get_variables_to_restore(exclude = exclude)

In [8]:
#Perform one-hot-encoding of the labels (Try one-hot-encoding within the load_batch function!)
one_hot_labels = slim.one_hot_encoding(labels, dataset.num_classes)

In [9]:
#Performs the equivalent to tf.nn.sparse_softmax_cross_entropy_with_logits but enhanced with checks
loss = tf.losses.softmax_cross_entropy(onehot_labels = one_hot_labels, logits = logits)
total_loss = tf.losses.get_total_loss()    #obtain the regularization losses as well

In [10]:
#Create the global step for monitoring the learning_rate and training.
global_step = get_or_create_global_step()

In [11]:
#Define your exponentially decaying learning rate
lr = tf.train.exponential_decay(
    learning_rate = initial_learning_rate,
    global_step = global_step,
    decay_steps = decay_steps,
    decay_rate = learning_rate_decay_factor,
    staircase = True
)

In [12]:
#Now we can define the optimizer that takes on the learning rate
optimizer = tf.train.AdamOptimizer(learning_rate = lr)

In [13]:
#Create the train_op.
train_op = slim.learning.create_train_op(total_loss, optimizer)

In [14]:
#State the metrics that you want to predict. We get a predictions that is not one_hot_encoded.
predictions = tf.argmax(end_points['Predictions'], 1)
probabilities = end_points['Predictions']
accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, labels)
metrics_op = tf.group(accuracy_update, probabilities)

In [15]:
#Now finally create all the summaries you need to monitor and group them into one summary op.
tf.summary.scalar('losses/Total_Loss', total_loss)
tf.summary.scalar('accuracy', accuracy)
tf.summary.scalar('learning_rate', lr)
my_summary_op = tf.summary.merge_all()

In [16]:
#Now we need to create a training step function that runs both the train_op, metrics_op and updates the global_step concurrently.
def train_step(sess, train_op, global_step):
    '''
    Simply runs a session for the three arguments provided and gives a logging on the time elapsed for each global step
    '''
    #Check the time for each sess run
    start_time = time.time()
    total_loss, global_step_count, _ = sess.run([train_op, global_step, metrics_op])
    time_elapsed = time.time() - start_time

    #Run the logging to print some results
    logging.info('global step %s: loss: %.4f (%.2f sec/step)', global_step_count, total_loss, time_elapsed)

    return total_loss, global_step_count

In [17]:
#Now we create a saver function that actually restores the variables from a checkpoint file in a sess
saver = tf.train.Saver(variables_to_restore)
def restore_fn(sess):
    return saver.restore(sess, checkpoint_file)

In [18]:
#Define your supervisor for running a managed session. 
#Do not run the summary_op automatically or else it will consume too much memory

sv = tf.train.Supervisor(logdir = log_dir, summary_op = None, init_fn = restore_fn)

In [19]:
#Run the managed session
with sv.managed_session() as sess:
    
    _num_steps_per_epoch, _num_batches_per_epoch = sess.run([num_steps_per_epoch, num_batches_per_epoch])    
    
    print("num_steps_per_epoch:", _num_steps_per_epoch)  
    print("num_batches_per_epoch:", _num_batches_per_epoch)
    print("num_epochs:", num_epochs)
    
    print("total steps:", _num_steps_per_epoch * num_epochs)
    
    for step in range(_num_steps_per_epoch * num_epochs):
        
        #At the start of every epoch, show the vital information:
        if step % _num_batches_per_epoch == 0:
            logging.info('Epoch %s/%s', step/_num_batches_per_epoch + 1, num_epochs)
            learning_rate_value, accuracy_value = sess.run([lr, accuracy])
            logging.info('Current Learning Rate: %s', learning_rate_value)
            logging.info('Current Streaming Accuracy: %s', accuracy_value)

            # optionally, print your logits and predictions for a sanity check that things are going fine.
            logits_value, probabilities_value, predictions_value, labels_value = sess.run([
                logits, 
                probabilities, 
                predictions, 
                labels
            ])
            
            print ('logits: \n', logits_value)
            print ('Probabilities: \n', probabilities_value)
            print ('predictions: \n', predictions_value)
            print ('Labels:\n:', labels_value)

        #Log the summaries every 10 step.
        if step % 100 == 0:
            loss, _ = train_step(sess, train_op, sv.global_step)
            summaries = sess.run(my_summary_op)
            sv.summary_computed(sess, summaries)

        #If not, simply run the training step
        else:
            loss, _ = train_step(sess, train_op, sv.global_step)

    #We log the final training loss and accuracy
    logging.info('Final Loss: %s', loss)
    logging.info('Final Accuracy: %s', sess.run(accuracy))

    #Once all the training has been done, save the log files and checkpoint model
    logging.info('Finished training! Saving model to disk now.')
    
    sv.saver.save(sess, sv.save_path, global_step = sv.global_step)

INFO:tensorflow:Restoring parameters from inception_files/models/inception_v1.ckpt
INFO:tensorflow:global_step/sec: inf
num_steps_per_epoch: 32
num_batches_per_epoch: 32
num_epochs: 30
total steps: 960
INFO:tensorflow:Epoch 1.0/30
INFO:tensorflow:Current Learning Rate: 0.0001
INFO:tensorflow:Current Streaming Accuracy: 0.0
logits: 
 [[  1.16444409e+00  -4.12756354e-01   1.05996571e-01  -8.94630611e-01
   -1.09754956e+00]
 [  6.28902674e-01  -1.17717719e+00   6.77061856e-01  -8.13799620e-01
    2.53358006e-01]
 [  1.38903427e+00   4.09420673e-03   4.11820740e-01  -6.91986144e-01
   -7.08162069e-01]
 [  5.21057904e-01  -6.57230437e-01   1.54520667e+00   1.96103960e-01
   -1.18151009e+00]
 [  9.32172596e-01  -9.95926857e-01  -1.39225185e-01   5.05344391e-01
   -2.08548740e-01]
 [  5.09815097e-01  -4.24284279e-01  -3.29638481e-01   1.13042183e-01
   -7.26304591e-01]
 [  7.97580481e-01   4.66814786e-01   5.10370314e-01  -7.63184354e-02
   -6.70315683e-01]
 [  9.05222952e-01  -1.76770866e-01

INFO:tensorflow:global step 1: loss: 2.2863 (5.70 sec/step)
INFO:tensorflow:global step 2: loss: 2.0033 (0.79 sec/step)
INFO:tensorflow:global step 3: loss: 2.0102 (0.80 sec/step)
INFO:tensorflow:global step 4: loss: 1.7372 (0.82 sec/step)
INFO:tensorflow:global step 5: loss: 1.6955 (0.95 sec/step)
INFO:tensorflow:global step 6: loss: 1.6995 (0.92 sec/step)
INFO:tensorflow:global step 7: loss: 1.4984 (0.97 sec/step)
INFO:tensorflow:global step 8: loss: 1.3247 (0.98 sec/step)
INFO:tensorflow:global step 9: loss: 1.3085 (0.96 sec/step)
INFO:tensorflow:global step 10: loss: 1.2769 (0.95 sec/step)
INFO:tensorflow:global step 11: loss: 1.2451 (0.93 sec/step)
INFO:tensorflow:global step 12: loss: 1.0480 (0.92 sec/step)
INFO:tensorflow:global step 13: loss: 1.1996 (0.90 sec/step)
INFO:tensorflow:global step 14: loss: 1.0302 (0.90 sec/step)
INFO:tensorflow:global step 15: loss: 1.2769 (0.91 sec/step)
INFO:tensorflow:global step 16: loss: 1.0194 (0.89 sec/step)
INFO:tensorflow:global step 17: l

INFO:tensorflow:global step 33: loss: 0.6917 (0.94 sec/step)
INFO:tensorflow:global step 34: loss: 0.6785 (0.94 sec/step)
INFO:tensorflow:global step 35: loss: 0.6874 (0.97 sec/step)
INFO:tensorflow:global step 36: loss: 0.6897 (0.97 sec/step)
INFO:tensorflow:global step 37: loss: 0.7031 (0.96 sec/step)
INFO:tensorflow:global step 38: loss: 0.7697 (0.96 sec/step)
INFO:tensorflow:global step 39: loss: 0.6187 (0.97 sec/step)
INFO:tensorflow:global step 40: loss: 0.7864 (0.96 sec/step)
INFO:tensorflow:global step 41: loss: 0.7764 (0.97 sec/step)
INFO:tensorflow:global step 42: loss: 0.7099 (0.97 sec/step)
INFO:tensorflow:global step 43: loss: 0.5645 (0.98 sec/step)
INFO:tensorflow:global step 44: loss: 0.6322 (0.98 sec/step)
INFO:tensorflow:global step 45: loss: 0.7672 (0.99 sec/step)
INFO:tensorflow:global step 46: loss: 0.8043 (1.00 sec/step)
INFO:tensorflow:global step 47: loss: 0.5256 (1.03 sec/step)
INFO:tensorflow:global step 48: loss: 0.5522 (1.01 sec/step)
INFO:tensorflow:global s

INFO:tensorflow:global step 65: loss: 0.6673 (0.98 sec/step)
INFO:tensorflow:global step 66: loss: 0.6451 (0.99 sec/step)
INFO:tensorflow:global step 67: loss: 0.5244 (0.99 sec/step)
INFO:tensorflow:global step 68: loss: 0.4471 (0.99 sec/step)
INFO:tensorflow:global step 69: loss: 0.5636 (0.99 sec/step)
INFO:tensorflow:global step 70: loss: 0.5529 (1.00 sec/step)
INFO:tensorflow:global step 71: loss: 0.5789 (0.98 sec/step)
INFO:tensorflow:global step 72: loss: 0.6985 (0.98 sec/step)
INFO:tensorflow:global step 73: loss: 0.6023 (1.00 sec/step)
INFO:tensorflow:global step 74: loss: 0.6449 (0.99 sec/step)
INFO:tensorflow:global step 75: loss: 0.7361 (0.99 sec/step)
INFO:tensorflow:global step 76: loss: 0.5248 (1.00 sec/step)
INFO:tensorflow:global step 77: loss: 0.4756 (0.99 sec/step)
INFO:tensorflow:global step 78: loss: 0.6366 (0.99 sec/step)
INFO:tensorflow:global step 79: loss: 0.4432 (0.99 sec/step)
INFO:tensorflow:global step 80: loss: 0.4399 (0.99 sec/step)
INFO:tensorflow:global s

INFO:tensorflow:global step 97: loss: 0.6750 (0.98 sec/step)
INFO:tensorflow:global step 98: loss: 0.4768 (0.99 sec/step)
INFO:tensorflow:global step 99: loss: 0.4821 (0.99 sec/step)
INFO:tensorflow:global step 100: loss: 0.3608 (0.99 sec/step)
INFO:tensorflow:global step 101: loss: 0.5483 (0.98 sec/step)
INFO:tensorflow:global step 102: loss: 0.4557 (1.00 sec/step)
INFO:tensorflow:global step 103: loss: 0.6364 (1.27 sec/step)
INFO:tensorflow:global step 104: loss: 0.5310 (0.96 sec/step)
INFO:tensorflow:global step 105: loss: 0.4976 (0.95 sec/step)
INFO:tensorflow:global step 106: loss: 0.4621 (0.96 sec/step)
INFO:tensorflow:global step 107: loss: 0.4980 (0.95 sec/step)
INFO:tensorflow:global step 108: loss: 0.4696 (0.98 sec/step)
INFO:tensorflow:global step 109: loss: 0.3503 (0.95 sec/step)
INFO:tensorflow:global step 110: loss: 0.3692 (0.96 sec/step)
INFO:tensorflow:global step 111: loss: 0.4383 (0.96 sec/step)
INFO:tensorflow:global step 112: loss: 0.4550 (0.97 sec/step)
INFO:tensor

...

INFO:tensorflow:global step 801: loss: 0.2649 (0.99 sec/step)
INFO:tensorflow:global step 802: loss: 0.3045 (1.00 sec/step)
INFO:tensorflow:global step 803: loss: 0.4602 (1.05 sec/step)
INFO:tensorflow:global step 804: loss: 0.3165 (1.02 sec/step)
INFO:tensorflow:global step 805: loss: 0.2888 (1.00 sec/step)
INFO:tensorflow:global step 806: loss: 0.3370 (1.00 sec/step)
INFO:tensorflow:global step 807: loss: 0.3277 (1.00 sec/step)
INFO:tensorflow:global step 808: loss: 0.3453 (0.99 sec/step)
INFO:tensorflow:global step 809: loss: 0.4272 (1.00 sec/step)
INFO:tensorflow:global step 810: loss: 0.3549 (1.00 sec/step)
INFO:tensorflow:global step 811: loss: 0.3889 (1.00 sec/step)
INFO:tensorflow:global step 812: loss: 0.3053 (1.01 sec/step)
INFO:tensorflow:global step 813: loss: 0.3332 (1.00 sec/step)
INFO:tensorflow:global step 814: loss: 0.3392 (1.00 sec/step)
INFO:tensorflow:global step 815: loss: 0.2654 (1.00 sec/step)
INFO:tensorflow:global step 816: loss: 0.3366 (0.99 sec/step)
INFO:ten

INFO:tensorflow:global step 833: loss: 0.2784 (1.01 sec/step)
INFO:tensorflow:global step 834: loss: 0.3750 (1.02 sec/step)
INFO:tensorflow:global step 835: loss: 0.3716 (1.01 sec/step)
INFO:tensorflow:global step 836: loss: 0.4168 (1.01 sec/step)
INFO:tensorflow:global step 837: loss: 0.3396 (1.01 sec/step)
INFO:tensorflow:global step 838: loss: 0.3784 (1.02 sec/step)
INFO:tensorflow:global step 839: loss: 0.3487 (1.04 sec/step)
INFO:tensorflow:global step 840: loss: 0.3301 (1.03 sec/step)
INFO:tensorflow:global step 841: loss: 0.2749 (1.01 sec/step)
INFO:tensorflow:global step 842: loss: 0.3746 (1.01 sec/step)
INFO:tensorflow:global step 843: loss: 0.3209 (1.01 sec/step)
INFO:tensorflow:global step 844: loss: 0.2689 (1.00 sec/step)
INFO:tensorflow:global step 845: loss: 0.3047 (1.03 sec/step)
INFO:tensorflow:global step 846: loss: 0.3289 (1.03 sec/step)
INFO:tensorflow:global step 847: loss: 0.4147 (1.04 sec/step)
INFO:tensorflow:global step 848: loss: 0.3926 (1.04 sec/step)
INFO:ten

INFO:tensorflow:global step 865: loss: 0.3937 (1.02 sec/step)
INFO:tensorflow:global step 866: loss: 0.2418 (1.03 sec/step)
INFO:tensorflow:global step 867: loss: 0.3434 (1.03 sec/step)
INFO:tensorflow:global step 868: loss: 0.2686 (1.04 sec/step)
INFO:tensorflow:global step 869: loss: 0.2937 (1.03 sec/step)
INFO:tensorflow:global step 870: loss: 0.2289 (1.03 sec/step)
INFO:tensorflow:global step 871: loss: 0.3215 (1.04 sec/step)
INFO:tensorflow:global step 872: loss: 0.3366 (1.05 sec/step)
INFO:tensorflow:global step 873: loss: 0.3695 (1.03 sec/step)
INFO:tensorflow:global step 874: loss: 0.3198 (1.04 sec/step)
INFO:tensorflow:global step 875: loss: 0.4289 (1.04 sec/step)
INFO:tensorflow:global step 876: loss: 0.2645 (1.01 sec/step)
INFO:tensorflow:global step 877: loss: 0.2674 (1.00 sec/step)
INFO:tensorflow:global step 878: loss: 0.4496 (1.01 sec/step)
INFO:tensorflow:global step 879: loss: 0.3168 (1.00 sec/step)
INFO:tensorflow:global step 880: loss: 0.3044 (1.02 sec/step)
INFO:ten

INFO:tensorflow:global step 897: loss: 0.3029 (1.01 sec/step)
INFO:tensorflow:global step 898: loss: 0.4251 (1.00 sec/step)
INFO:tensorflow:global step 899: loss: 0.3839 (1.00 sec/step)
INFO:tensorflow:global step 900: loss: 0.3651 (1.00 sec/step)
INFO:tensorflow:global step 901: loss: 0.3921 (1.01 sec/step)
INFO:tensorflow:global step 902: loss: 0.3474 (1.01 sec/step)
INFO:tensorflow:global step 903: loss: 0.3653 (1.00 sec/step)
INFO:tensorflow:global step 904: loss: 0.3305 (1.04 sec/step)
INFO:tensorflow:global step 905: loss: 0.2881 (1.02 sec/step)
INFO:tensorflow:global step 906: loss: 0.3001 (1.04 sec/step)
INFO:tensorflow:global step 907: loss: 0.3271 (1.04 sec/step)
INFO:tensorflow:global step 908: loss: 0.2918 (1.02 sec/step)
INFO:tensorflow:global step 909: loss: 0.3336 (1.02 sec/step)
INFO:tensorflow:global step 910: loss: 0.3486 (1.03 sec/step)
INFO:tensorflow:global step 911: loss: 0.2786 (1.03 sec/step)
INFO:tensorflow:global step 912: loss: 0.2890 (1.01 sec/step)
INFO:ten

INFO:tensorflow:global step 929: loss: 0.2628 (0.99 sec/step)
INFO:tensorflow:global step 930: loss: 0.3355 (0.99 sec/step)
INFO:tensorflow:global step 931: loss: 0.2744 (1.00 sec/step)
INFO:tensorflow:global step 932: loss: 0.4466 (1.00 sec/step)
INFO:tensorflow:global step 933: loss: 0.4445 (1.01 sec/step)
INFO:tensorflow:global_step/sec: 0.958323
INFO:tensorflow:global step 934: loss: 0.2622 (1.02 sec/step)
INFO:tensorflow:global step 935: loss: 0.3562 (1.02 sec/step)
INFO:tensorflow:global step 936: loss: 0.3189 (1.06 sec/step)
INFO:tensorflow:global step 937: loss: 0.4219 (1.03 sec/step)
INFO:tensorflow:global step 938: loss: 0.2803 (1.00 sec/step)
INFO:tensorflow:global step 939: loss: 0.3352 (1.00 sec/step)
INFO:tensorflow:global step 940: loss: 0.3314 (1.00 sec/step)
INFO:tensorflow:global step 941: loss: 0.3260 (1.01 sec/step)
INFO:tensorflow:global step 942: loss: 0.2652 (1.03 sec/step)
INFO:tensorflow:global step 943: loss: 0.3502 (1.01 sec/step)
INFO:tensorflow:global step 